In [2]:
import numpy as np
import keras
from keras.models import Model, load_model
from keras.layers import Dense, Input, Concatenate, Dropout, Add, Lambda
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
import os
from keras.engine.topology import Layer
import pickle
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
os.environ["CUDA_VISIBLE_DEVICES"]="6"



Using TensorFlow backend.


In [ ]:
DATA = "ARTINT"

In [3]:
!nvidia-smi

Sun Jan 13 15:19:08 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0  On |                  N/A |
| 35%   64C    P2    74W / 250W |   9081MiB / 11172MiB |     35%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 23%   36C    P8    16W / 250W |    506MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [11]:
#load train data
train_img_filename =  pickle.load(open(DATA+"_train_img_filename_for_img_modal_vector.p", "rb"))
train_img_feature_vector = np.load(DATA+'_train_img_modal_vector.npy')
train_caption_feature_vector = pickle.load(open(DATA+"_train_image_caption_feature_vector.p", "rb"))
train_text_filename = pickle.load(open(DATA+"_train_fulltext_filename_final.p", "rb"))
train_text_feature_vector =  np.load(DATA+"_train_full_text_modality_feature.npy")
train_bib_feature_vector =  pickle.load(open(DATA+"_train_only_bib_feature_vector_final.p", "rb"))



In [12]:
#load test data
test_img_filename = pickle.load(open(DATA+"_test_img_filename_for_img_modal_vector.p", "rb"))
test_img_feature_vector = np.load(DATA+'_test_img_modal_vector.npy')
test_caption_feature_vector = pickle.load(open(DATA+"_test_image_caption_feature_vector.p", "rb"))
test_text_filename = pickle.load(open(DATA+"_test_fulltext_filename_final.p", "rb"))
test_text_feature_vector = np.load(DATA+"_test_full_text_modality_feature.npy")
test_bib_feature_vector = pickle.load(open(DATA+"_test_only_bib_feature_vector_final.p", "rb"))


In [13]:
#print(len(train_img_filename))
#print(len(train_text_filename))
#print(train_caption_feature_vector.keys())
#print(train_bib_feature_vector.keys())

In [14]:
fv_img = []
fv_img_caption = []
fv_bib = []
train_labels = []
for i in range(len(train_img_filename)):
    for j in range(len(train_text_filename)):
        if train_img_filename[i] in train_text_filename[j]:
            if train_text_filename[j] not in train_caption_feature_vector.keys():
                continue
            if train_text_filename[j] not in train_bib_feature_vector.keys():
                    continue
            fv_img.append(train_img_feature_vector[i])
            fv_img_caption.append(train_caption_feature_vector[train_text_filename[j]])
            fv_bib.append(train_bib_feature_vector[train_text_filename[j]])
            label = 1.0
            if 'neg_' in train_img_filename[i]:
                label = 0.0
            train_labels.append(label)

fv_img = np.array(fv_img)
fv_img_caption = np.array(fv_img_caption)
fv_bib = np.array(fv_bib)

In [15]:
fv_img_test = []
fv_img_test_caption = []
fv_bib_test = []
test_labels = []
for i in range(len(test_img_filename)):
    for j in range(len(test_text_filename)):
        if test_img_filename[i] in test_text_filename[j]:
            if test_text_filename[j] not in test_caption_feature_vector.keys():
                continue
            if test_text_filename[j] not in test_bib_feature_vector.keys():
                print(test_text_filename[j])
            fv_img_test.append(test_img_feature_vector[i])
            fv_img_test_caption.append(test_caption_feature_vector[test_text_filename[j]])
            fv_bib_test.append(test_bib_feature_vector[test_text_filename[j]])
            label = 1.0
            if 'neg_' in test_img_filename[i]:
                label = 0.0
            test_labels.append(label)
            
fv_img_test = np.array(fv_img_test)
fv_img_test_caption = np.array(fv_img_test_caption)
fv_bib_test = np.array(fv_bib_test)

In [19]:
print(fv_img.shape)
print(fv_img_caption.shape)
print(fv_bib.shape)
print(fv_img_test.shape)
print(fv_img_test_caption.shape)
print(fv_bib_test.shape)


(3056, 4096)
(3056, 5716)
(3056, 9000)
(768, 4096)
(768, 5716)
(768, 9000)


In [21]:
#model

IMG_FEATURES = Input(shape = (4096,))
IMG_CAPTIONS = Input(shape = (5716,))
BIB = Input(shape = (9000,))

B = Concatenate(axis = 1)([IMG_FEATURES, IMG_CAPTIONS, BIB])

P = Dense(512, activation = 'relu')(B)

alpha = Dense(3, activation = 'softmax')(P)

F = Lambda(lambda x : alpha[:,0:1]*IMG_FEATURES)(alpha)
G = Lambda(lambda x : alpha[:,1:2]*IMG_CAPTIONS)(alpha)
I = Lambda(lambda x : alpha[:,2:3]*BIB)(alpha)
Y = Concatenate(axis = -1)([F,G,I])

Y = Dense(2000, activation = 'relu')(Y)
Y = Dropout(rate = 0.2)(Y)

Y = Dense(512, activation = 'relu')(Y)		#, kernel_regularizer = regularizers.l2(0.01)
Y = Dropout(rate = 0.25)(Y)

Y = Dense(512, activation = 'relu')(Y)
Y = Dropout(rate = 0.2)(Y)

Y = Dense(1, activation = 'sigmoid')(Y)

model = Model(inputs = [IMG_FEATURES, IMG_CAPTIONS, BIB], outputs = Y)

In [ ]:
checkpoint = ModelCheckpoint(DATA+'model-img-caption-bib-best.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

results = model.fit( [fv_img,fv_img_caption,fv_bib],train_labels, epochs= 20, batch_size = 64,validation_data=([fv_img_test,fv_img_test_caption,fv_bib_test],test_labels),callbacks=[checkpoint])


Train on 3056 samples, validate on 768 samples
Epoch 1/20
3056/3056 [==============================] - 10s 3ms/step - loss: 0.3809 - acc: 0.8331 - val_loss: 0.2746 - val_acc: 0.8828

Epoch 00001: val_loss improved from inf to 0.27464, saving model to model-img-caption-bib-best.h5
Epoch 2/20
3056/3056 [==============================] - 2s 657us/step - loss: 0.1417 - acc: 0.9444 - val_loss: 0.3560 - val_acc: 0.8867

Epoch 00002: val_loss did not improve from 0.27464
Epoch 3/20
3056/3056 [==============================] - 2s 661us/step - loss: 0.0836 - acc: 0.9627 - val_loss: 0.5270 - val_acc: 0.8893

Epoch 00003: val_loss did not improve from 0.27464
Epoch 4/20
3056/3056 [==============================] - 2s 676us/step - loss: 0.0721 - acc: 0.9663 - val_loss: 0.5731 - val_acc: 0.8711

Epoch 00004: val_loss did not improve from 0.27464
Epoch 5/20
3056/3056 [==============================] - 2s 703us/step - loss: 0.0729 - acc: 0.9666 - val_loss: 0.5852 - val_acc: 0.8828

Epoch 00005: val_l

In [ ]:
predicted = model.predict([fv_img_test,fv_img_test_caption,fv_bib_test])

In [ ]:
for i in range(len(predicted)):
    if predicted[i] >= 0.5:
        predicted[i] = 1.0
    else :
        predicted[i] = 0.0

In [ ]:
print("fscore",f1_score(test_labels, predicted, average="macro"))
print("precision",precision_score(test_labels, predicted, average="macro"))
print("recall",recall_score(test_labels, predicted, average="macro"))    

x = classification_report(test_labels,predicted)
print(x)

In [ ]:
model.save(DATA+'image-caption-bib-model')